# Scala Experiment

We set the Anonymous AWS Credentials Provider as we don't need to authenticate for aws open data


In [2]:
%%init_spark
launcher.master = "spark://spark-master:7077"
launcher.num_executors = 2
launcher.executor_cores = 2
launcher.executor_memory = '4G'
launcher.packages = ["com.amazon.deequ:deequ:1.2.2-spark-3.0",
                    "org.apache.hadoop:hadoop-aws:3.2.0"]
launcher.conf.set("spark.hadoop.fs.s3a.aws.credentials.provider", 
                  "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider")

In [3]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://7847ea2e55be:4040
SparkContext available as 'sc' (version = 3.1.1, master = spark://spark-master:7077, app id = app-20210522141525-0006)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@94011f


In [4]:
spark.version

res1: String = 3.1.1


In [18]:
import org.apache.spark.sql.SparkSession
import com.amazon.deequ.VerificationSuite
import com.amazon.deequ.checks.{Check, CheckLevel, CheckStatus}
import com.amazon.deequ.constraints.ConstraintStatus
import java.net.{URI, URLEncoder}
import org.apache.hadoop.fs.{FileSystem, Path, RemoteIterator, LocatedFileStatus}
import scala.collection.mutable.ListBuffer

import org.apache.spark.sql.SparkSession
import com.amazon.deequ.VerificationSuite
import com.amazon.deequ.checks.{Check, CheckLevel, CheckStatus}
import com.amazon.deequ.constraints.ConstraintStatus
import java.net.{URI, URLEncoder}
import org.apache.hadoop.fs.{FileSystem, Path, RemoteIterator, LocatedFileStatus}
import scala.collection.mutable.ListBuffer


Check Raw TLC bucket data

In [11]:
val rawS3Dir = "s3a://nyc-tlc/"
//val fileSystem = FileSystem.get(URI.create(rawS3Dir), new Configuration())
val fileSystem = FileSystem.get(URI.create(rawS3Dir), spark.sparkContext.hadoopConfiguration)
val itemlist = fileSystem.listFiles(new Path(rawS3Dir), true)       

rawS3Dir: String = s3a://nyc-tlc/
fileSystem: org.apache.hadoop.fs.FileSystem = S3AFileSystem{uri=s3a://nyc-tlc, workingDir=s3a://nyc-tlc/user/jovyan, inputPolicy=normal, partSize=104857600, enableMultiObjectsDelete=true, maxKeys=5000, readAhead=65536, blockSize=33554432, multiPartThreshold=2147483647, serverSideEncryptionAlgorithm='NONE', blockFactory=org.apache.hadoop.fs.s3a.S3ADataBlocks$DiskBlockFactory@13747edc, metastore=NullMetadataStore, authoritative=false, useListV1=false, magicCommitter=false, boundedExecutor=BlockingThreadPoolExecutorService{SemaphoredDelegatingExecutor{permitCount=25, available=25, waiting=0}, activeCount=0}, unboundedExecutor=java.util.concurrent.ThreadPoolExecutor@5293c779[Running, pool size = 0, active threads = 0, queued tasks = 0, completed tasks = 0],...


In [12]:
val limit = 10000
var cnt = 1

limit: Int = 10000
cnt: Int = 1


In [13]:
def loopy(iter: RemoteIterator[LocatedFileStatus]) : ListBuffer[String] = {

    var files = new ListBuffer[String]()
            
    while(cnt <  limit) {
        if (iter.hasNext) {
            val uri = iter.next.getPath.toUri.toString
            files += uri
            cnt += 1
        } else {
            cnt += 1
        }
    }

    return files 
}

loopy: (iter: org.apache.hadoop.fs.RemoteIterator[org.apache.hadoop.fs.LocatedFileStatus])scala.collection.mutable.ListBuffer[String]


In [14]:
val fileList = loopy(itemlist).toList

fileList: List[String] = List(s3a://nyc-tlc/misc/FOIL_2017-12-04.csv, s3a://nyc-tlc/misc/FOIL_fhv_trips_june17.csv, s3a://nyc-tlc/misc/FOIL_medallion_trips_june17.csv, s3a://nyc-tlc/misc/FOIL_weekly_trips_apps.csv, s3a://nyc-tlc/misc/FOIL_weekly_trips_yellow.csv, s3a://nyc-tlc/misc/FOIL_yellow_trips_deadhead.csv, s3a://nyc-tlc/misc/FOIL_yellow_trips_deadhead_apr13.csv, s3a://nyc-tlc/misc/foil26707_1.csv, s3a://nyc-tlc/misc/foil26707_2.csv, s3a://nyc-tlc/misc/foil26707_3.csv, s3a://nyc-tlc/misc/foil26707_4.csv, s3a://nyc-tlc/misc/shared_rides_by_puzone_monthly.csv, s3a://nyc-tlc/misc/taxi%20_zone_lookup.csv, s3a://nyc-tlc/misc/taxi_zones.html, s3a://nyc-tlc/misc/taxi_zones.zip, s3a://nyc-tlc/misc/uber_nyc_data.csv, s3a://nyc-tlc/trip%20data/fhv_tripdata_2015-01.csv, s3a://nyc-tlc/trip%20...


Check the specific trip data

Load the green trip data

In [42]:
val green_trip_data = "s3a://nyc-tlc/trip data/green_tripdata_2018*.csv"
val taxi_test = spark.read.option("header", true).csv(green_trip_data)

green_trip_data: String = s3a://nyc-tlc/trip data/green_tripdata_2018*.csv
taxi_test: org.apache.spark.sql.DataFrame = [VendorID: string, lpep_pickup_datetime: string ... 17 more fields]


In [43]:
taxi_test.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- ehail_fee: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- trip_type: string (nullable = true)



In [45]:
// try deequ test
val verificationResult = VerificationSuite()
    .onData(taxi_test)
    .addCheck(
        Check(CheckLevel.Error, "unit testing my data")
            .isComplete("lpep_pickup_datetime")
            .isNonNegative("fare_amount")
        )
    .run()

verificationResult: com.amazon.deequ.VerificationResult = VerificationResult(Error,Map(Check(Error,unit testing my data,List(CompletenessConstraint(Completeness(lpep_pickup_datetime,None)), ComplianceConstraint(Compliance(fare_amount is non-negative,COALESCE(CAST(fare_amount AS DECIMAL(20,10)), 0.0) >= 0,None)))) -> CheckResult(Check(Error,unit testing my data,List(CompletenessConstraint(Completeness(lpep_pickup_datetime,None)), ComplianceConstraint(Compliance(fare_amount is non-negative,COALESCE(CAST(fare_amount AS DECIMAL(20,10)), 0.0) >= 0,None)))),Error,List(ConstraintResult(CompletenessConstraint(Completeness(lpep_pickup_datetime,None)),Success,None,Some(DoubleMetric(Column,Completeness,lpep_pickup_datetime,Success(1.0)))), ConstraintResult(ComplianceConstraint(Compliance(fare_amou...


In [46]:
if (verificationResult.status == CheckStatus.Success) {
  println("The data passed the test, everything is fine!")
} else {
  println("We found errors in the data:\n")

  val resultsForAllConstraints = verificationResult.checkResults
    .flatMap { case (_, checkResult) => checkResult.constraintResults }

  resultsForAllConstraints
    .filter { _.status != ConstraintStatus.Success }
    .foreach { result => println(s"${result.constraint}: ${result.message.get}") }
}

We found errors in the data:

ComplianceConstraint(Compliance(fare_amount is non-negative,COALESCE(CAST(fare_amount AS DECIMAL(20,10)), 0.0) >= 0,None)): Value: 0.9973816047886623 does not meet the constraint requirement!


In [ ]:
// stop the session
spark.stop()